In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('.\datasets\hfi_cc_2018.csv')
data = data[[
'year',
'countries',
'region',
'pf_rank',
'pf_score',
#'pf_rol_civil',
'pf_ss_homicide',
'pf_ss_disappearances',
'pf_movement',
'pf_religion_estop',
#'pf_association_political_establish',
'pf_association',
'pf_expression_internet',
'pf_identity_divorce',
'ef_government_tax',
'ef_government',
'ef_money_currency',
'ef_trade_tariffs_mean',
'ef_trade_movement_visit']]
before_2016_data = data[data['year'] == 2016]
before_2016_data['year'].value_counts()
temp_data = before_2016_data[before_2016_data.isnull().sum(axis = 1) < 2]
temp_data.isnull().sum(axis = 1)

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
9      0
10     0
12     0
13     0
15     0
17     0
18     0
20     0
21     1
22     0
23     0
24     0
25     0
26     0
27     0
29     0
30     0
31     0
32     0
33     0
35     0
36     0
      ..
123    0
124    0
125    0
126    0
127    0
130    0
131    0
132    0
133    0
134    0
135    0
136    0
139    0
140    0
142    1
143    0
144    0
145    0
147    0
149    0
150    0
151    0
152    0
153    0
154    0
155    0
156    0
157    0
158    0
161    1
Length: 136, dtype: int64

#### Kood millega lisasin GDP teise datasetti
data_2 = pd.read_csv('.\datasets\Dataset2.csv')
gdp_data = pd.read_csv('.\datasets\gdp_data.csv')

gdp_data = gdp_data.drop(columns=['Country Code', 'Series Name', 'Series Code', '1990 [YR1990]', '2000 [YR2000]', '2019 [YR2019]', '2017 [YR2017]', '2018 [YR2018]'])

for year in range(2010, 2017, 1):
    print(year)
    gdp_data.rename(columns={(str(year) + " [YR" + str(year) + "]"): year}, inplace=True)

gdp_data = gdp_data.melt(id_vars=["Country Name"], 
        var_name="Year", 
        value_name="GDP")

#gdp_data = gdp_data.loc[gdp_data['Country Name'] in data_2['Country Name']]

gdp_data = gdp_data.reset_index()
for i in range(len(gdp_data)):
    if (gdp_data.loc[i]['Country Name']) not in set(data_2['Country Name']):
        gdp_data = gdp_data.drop(i, axis=0)




data_2 = data_2.merge(gdp_data, on=['Country Name', 'Year'], how='inner').drop(columns=['index'])
#data_2.to_csv("dataset2.csv", index=False)

In [4]:
data_2 = pd.read_csv('.\datasets\dataset2.csv')

before_2016_data = data_2[data_2['Year'] == 2010]
before_2016_data['Year'].value_counts()
temp_data = before_2016_data[before_2016_data.isnull().sum(axis = 1) < 2]
temp_data.isnull().sum(axis = 1)

1841    0
1846    0
1849    1
1850    0
1852    0
1853    0
1854    0
1855    1
1857    0
1858    1
1859    0
1860    0
1861    0
1864    1
1866    0
1867    0
1868    0
1871    0
1872    0
1874    0
1875    0
1877    0
1878    1
1880    0
1881    0
1884    0
1885    1
1886    0
1889    0
1890    0
       ..
2048    0
2050    0
2052    0
2053    0
2057    0
2058    0
2059    0
2061    0
2062    0
2064    1
2066    1
2072    0
2073    0
2076    0
2077    0
2078    0
2079    0
2081    0
2082    0
2083    0
2088    0
2090    0
2091    0
2092    1
2093    0
2095    1
2099    1
2100    0
2102    0
2103    0
Length: 152, dtype: int64